**Introduction**
From the meta data generate shape files and upload them to google earth engine as feature collections.And for each feature collection run the codes in the notebook

In [ ]:
import ee
from google.colab import auth

auth.authenticate_user()
ee.Initialize(project='ee-angelanyangate4565')


##TREE COVER COLUMNS; "tree_cover_area_2000,tree_cover_area_2005 tree_cover_area_2010,tree_cover_area_2015 AND tree_cover_area_2020

In [ ]:
#Loading the Polygon as a feature collection from earth engine
plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/verra_asia')

# Load the land cover image and apply a land mask
landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2010').updateMask(landmask) # CHANGE YEARS 2000,2005,2010,2015,2020

# Defining class codes and corresponding classes of interest
classCodes = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
              125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]

# Mask the land cover image
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

# Function to calculate area for each class within each polygon
def calculate_area(feature):
    total_area = ee.Number(0)

    def calculate_class_area(class_code, total):
        class_area = maskedLandCover.eq(ee.Number(class_code)) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            ).get('remapped')

        area_value = ee.Number(class_area).divide(1e6)  # Convert to km^2
        return ee.Number(total).add(area_value)

    total_area = ee.List(classCodes).iterate(calculate_class_area, total_area)
    return feature.set('tree_cover_area_2010', total_area)

# Map the function over the feature collection
area_results = plantp.map(calculate_area)

# Export the results to Google Drive as a CSV file
task = ee.batch.Export.table.toDrive(
    collection=area_results,
    description='asia',
    folder='EarthEngineResults',
    fileNamePrefix='new_2010_asia_land_cover_areas_with_total',
    fileFormat='CSV'
)
task.start()

print("Exporting...")


import time

while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)

print('Done with export.')




Exporting...
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling for task (id: MO57ZA5HFJUHKPVVQ4ZPLN6S).
Polling

##CHANGE COLUMNS EXTRACTION ("permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees")

In [ ]:

plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/combined_shapefile')

# Defining the landmask and land cover images
landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC').updateMask(landmask)

# Defining the class codes and corresponding names of interest
classCodes = [208, 240, 248, 245]
classesOfInterest = ["permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees"]

maskedLandCover = landCover.remap(classCodes, classCodes, 255)  # no data regions

# Defining the function to calculate area for each class within each polygon
def calculate_area(feature):

    areas = {}


    for class_code, class_name in zip(classCodes, classesOfInterest):
        current_class_area = maskedLandCover.eq(class_code) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            )

        area_value = ee.Number(current_class_area.get('remapped', 0)).divide(1e6)  # Convert to km^2

        areas[class_name] = area_value


    return feature.set(areas)


area_results = plantp.map(calculate_area)


print('Area Results:', area_results.first().getInfo())

# Exporting and saving  the results as csv to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=area_results,
    description='atlas_land_cover_areas_2020',
    folder='EarthEngineResults',
    fileNamePrefix='atlas_verra_aggregated_land_cover_areas_with_total',
    fileFormat='CSV'
)
task.start()

print('Exporting to Google Drive...')

# Monitor the export task status
import time

while task.active():
    print('Waiting for task to complete...')
    time.sleep(30)

print('Export completed.')


Area Results: {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-76.44429118867342, -6.391479281140563], [-76.44423328468247, -6.391550558656469], [-76.44399690196524, -6.391746792331933], [-76.44388541885489, -6.391782487957311], [-76.44355997236592, -6.391568414973254], [-76.44376502051954, -6.39101999647197], [-76.44401473219258, -6.3911982905630165], [-76.4442065393853, -6.391390096530505], [-76.44426444338811, -6.391399010372884], [-76.44429118867342, -6.391479281140563]]]}, 'id': '00000000000000000000', 'properties': {'Name': 'Polygon108', 'cropland_gain_from_trees': 0, 'cropland_loss_to_tree': 0, 'extrude': 0, 'permanent_water': 0, 'short_vegetation_after_tree_loss': 0, 'tessellate': -1, 'visibility': -1}}
Exporting to Google Drive...
Waiting for task to complete...
Export completed.


##BUILT AREA CHARACTERISTICS EXTRACTION(BUILT_AREA)

In [ ]:


# Loading the Settlement Characteristics layer for 2018
builtImage = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics')


plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/verra_asia')

# Defining the number of patches due to the size of some polygon collections that causes computational timeout
patch_size = 10000
total_polygons = plantp.size()
num_patches = total_polygons.divide(patch_size).ceil()


def calculate_built_area(feature):
    polygon = feature.geometry()
    built_area = builtImage.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=10,
        maxPixels=1e9
    ).get('built_characteristics')

    built_area = ee.Number(built_area).max(0)
    return feature.set('built_area', built_area)

#Defing the function  processing each patch
def process_patch(patch_index):
    start = ee.Number(patch_index).multiply(patch_size)
    end = start.add(patch_size)

    patch = plantp.toList(patch_size, start)
    patch_fc = ee.FeatureCollection(patch)

    processed_patch = patch_fc.map(calculate_built_area)
    return processed_patch


patch_indices = ee.List.sequence(0, num_patches.subtract(1))
patches = patch_indices.map(process_patch)


processed_polygons = ee.FeatureCollection(patches).flatten()

first_feature = processed_polygons.first().getInfo()
print('First feature with built area:', first_feature)

# Exporting and saving  the results as csv to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=processed_polygons,
    description='New_verra_BuiltAreaStats_2018_PATCHED',
    fileFormat='CSV'
)
task.start()

print("Export started...")

# Monitoring the export task status
while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


First feature with built area: {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '0_0000000000000000257a', 'properties': {'built_area': 0, 'country': '', 'host_name': 'Verra', 'project_id': 'Verra_Asia_1', 'site_id_re': 'asi_reforest_site_837', 'site_sqkm': 0, 'url': 'https://registry.verra.org/app/projectDetail/CCB/2374'}}
Export started...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for task (id: A7ZO7NFB3YNYM2WILDZZHSAS)...
Polling for t

##ROAD NETWORKS COLUMNS(total_road_length_km)

In [ ]:

import time

In [ ]:


polygons = ee.FeatureCollection('projects/ee-forest-monitoring/assets/verra_asia')

# Loading all road NASA feature collections
roadsAfrica = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-africa')
roadsAmericas = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-americas')
roadsAsia = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-asia')
roadsEurope = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-europe')
roadsOceaniaEast = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-east')
roadsOceaniaWest = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-west')

# Merging all road feature collections
roads = roadsAfrica.merge(roadsAmericas).merge(roadsAsia).merge(roadsEurope).merge(roadsOceaniaEast).merge(roadsOceaniaWest)

def calculate_road_length(polygon):
    # Intersecting roads with the polygon area
    intersecting_roads = roads.filterBounds(polygon.geometry())

    # Clipping the intersecting roads to the polygon geometry
    def clip_and_calculate_length(road):
        road_geom = road.geometry()
        polygon_geom = polygon.geometry()
        clipped = road_geom.intersection(polygon_geom, ee.ErrorMargin(1))
        return ee.Feature(clipped).set('length', clipped.length())

    clipped_roads = intersecting_roads.map(clip_and_calculate_length)

    # Calculating the total length of the clipped road segments
    road_length_sum = clipped_roads.reduceColumns(
        reducer=ee.Reducer.sum(),
        selectors=['length']
    ).get('sum')

    # Counting the number of intersecting roads
    intersecting_roads_count = intersecting_roads.size()

    # Setting properties for the polygon feature
    return polygon.set({
        'total_road_length_km': ee.Number(road_length_sum).divide(1000),  # Convert to kilometers
        'intersecting_roads_count': intersecting_roads_count
    })

# Map the function over the Feature Collection
polygons_with_road_length = polygons.map(calculate_road_length)


first_feature = polygons_with_road_length.first().getInfo()
print('First feature with road length and count:', first_feature)
# Exporting and saving  the results as csv to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=polygons_with_road_length,
    description='new_asia_with_road_length',
    fileFormat='CSV'
)
task.start()

print("Export started...")


while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


First feature with road length and count: {'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '0000000000000000257a', 'properties': {'country': '', 'host_name': 'Verra', 'intersecting_roads_count': 0, 'project_id': 'Verra_Asia_1', 'site_id_re': 'asi_reforest_site_837', 'site_sqkm': 0, 'total_road_length_km': 0, 'url': 'https://registry.verra.org/app/projectDetail/CCB/2374'}}
Export started...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for task (id: BZSYT3RMLJZ6KAQD3SBN2SJS)...
Polling for ta

##GLOBAL FOREST LOSS GROUPS EXTRACTION,using the python code in the "data_analysis.ipny" create the colun s "loss_pre_5","loss_post_3",loss_post_5"

In [ ]:

plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/verra_asia')
gfc2017 = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')



#Defining a Function to calculate forest loss for each polygon
def calculate_loss(feature):
    loss_image = gfc2017.select(['loss'])
    loss_area_image = loss_image.multiply(ee.Image.pixelArea())
    loss_year = gfc2017.select(['lossyear'])


    loss_by_year = loss_area_image.addBands(loss_year).reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    )

    # Setting the loss data as properties of the feature
    return feature.set(loss_by_year)


loss_results = plantp.map(calculate_loss)

first_result = loss_results.first().getInfo()
print(first_result)

# Exporting and saving the results as CSV file to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=loss_results,
    description='forest_loss_by_polygon',
    fileFormat='CSV'
)
task.start()

print("Export started...")


while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


{'type': 'Feature', 'geometry': {'type': 'MultiPoint', 'coordinates': []}, 'id': '0000000000000000257a', 'properties': {'country': '', 'groups': [], 'host_name': 'Verra', 'project_id': 'Verra_Asia_1', 'site_id_re': 'asi_reforest_site_837', 'site_sqkm': 0, 'url': 'https://registry.verra.org/app/projectDetail/CCB/2374'}}
Export started...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)...
Polling for task (id: WQXTIXD3ZZAEFR6BD3BB66MK)..

##NDVI per MONTH

In [ ]:



geometry = ee.FeatureCollection("projects/ee-forest-monitoring/assets/verra_asia")


geometry = geometry.map(lambda feature: feature.simplify(30))

# Loading the Sentinel-2 Image Collection for the year 2023
S2 = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2023-01-01', '2023-12-31') \
  .filterBounds(geometry)


def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def reduce_to_monthly(image_collection):
    monthly = image_collection.select('NDVI').mean().rename('NDVI')
    return monthly


def calculate_and_export_monthly_ndvi(month):

    monthly_s2 = S2.filter(ee.Filter.calendarRange(month, month, 'month')).map(calculate_ndvi)


    monthly_ndvi = reduce_to_monthly(monthly_s2)

    mean_ndvi = monthly_ndvi.reduceRegions(
        collection=geometry,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    # Exporting and saving the results to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=mean_ndvi,
        description=f'monthly_atlas_{month}',
        fileFormat='CSV'
    )
    task.start()

# Looping through each month and process the data
for month in range(1, 13):
    calculate_and_export_monthly_ndvi(month)
    print(f'Exporting month {month}...')

tasks = ee.batch.Task.list()
while any([task.active() for task in tasks]):
    print('Waiting for all tasks to complete...')
    time.sleep(30)

print('All tasks completed.')


/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Exporting month 1...
Exporting month 2...
Exporting month 3...
Exporting month 4...
Exporting month 5...
Exporting month 6...
Exporting month 7...
Exporting month 8...
Exporting month 9...
Exporting month 10...
Exporting month 11...
Exporting month 12...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
All tasks completed.
